In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Final_Project/py-files/')

In [ ]:
%%bash
pip install -qqq -U langchain-huggingface
pip install -qqq -U langchain
pip install -qqq -U langchain-community
pip install -qqq -U faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 19.6 MB/s eta 0:00:00


In [ ]:
# imports
import numpy as np
import pandas as pd
from langchain.vectorstores import FAISS

In [ ]:
# import os
# from google.colab import userdata # we stored our access token as a colab secret

# os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('LLM_TOKEN')

# Load the data to create the FAISS Vector Store with

Now that we have the data, we have to load it in a format LangChain can understand. For this, Langchain has [loaders](https://python.langchain.com/docs/modules/data_connection/document_loaders/). There's loaders for CSV, text, PDF, and a host of other formats. You're not restricted to just text here.

## Load ISLP

In [ ]:
pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 4.3 MB/s eta 0:00:00


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/drive/MyDrive/Final_Project/data/books/ISLP.pdf")
islp_pages = loader.load_and_split()

## Load Frankenstein

In [ ]:
from langchain.document_loaders import TextLoader

loader = TextLoader("/content/drive/MyDrive/Final_Project/data/books/frankenstein.txt")
frankenstein_documents = loader.load()

# Load 1984

In [ ]:
# faiss_index_book3_1984
from langchain.document_loaders import TextLoader

loader = TextLoader("/content/drive/MyDrive/Final_Project/data/books/1984.txt")
nineteen_eighty_four_documents = loader.load()

## Splitting the document
Obviously, a whole book is a lot to digest. This is made easier by [splitting](https://python.langchain.com/docs/modules/data_connection/document_transformers/) the document into chunks. You can split it by paragraphs, sentences, or even individual words, depending on what you want to analyse. In Langchain, we have different tools like the RecursiveCharacterTextSplitter (say that five times fast!) that understand the structure of text and help you break it down into manageable chunks.

Check out [this website](https://chunkviz.up.railway.app/) to help visualise the splitting process.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=800,
                                               chunk_overlap=150)

In [ ]:
frankenstein_docs = text_splitter.split_documents(frankenstein_documents)

In [ ]:
nineteen_eighty_four_docs = text_splitter.split_documents(nineteen_eighty_four_documents)

## Test what the splitter did.

In [ ]:
frankenstein_docs[400].metadata['source']

'/content/drive/MyDrive/Final_Project/data/books/frankenstein.txt'

In [ ]:
frankenstein_docs[400].page_content

'“Felix had accidentally been present at the trial; his horror and\nindignation were uncontrollable when he heard the decision of the\ncourt. He made, at that moment, a solemn vow to deliver him and then\nlooked around for the means. After many fruitless attempts to gain\nadmittance to the prison, he found a strongly grated window in an\nunguarded part of the building, which lighted the dungeon of the\nunfortunate Muhammadan, who, loaded with chains, waited in despair the\nexecution of the barbarous sentence. Felix visited the grate at night\nand made known to the prisoner his intentions in his favour. The Turk,\namazed and delighted, endeavoured to kindle the zeal of his deliverer\nby promises of reward and wealth. Felix rejected his offers with'

In [ ]:
nineteen_eighty_four_docs[400].page_content

'round the lions at the base of the monument and joined in the rush.\nWinston followed. As he ran, he gathered from some shouted remarks that\na convoy of Eurasian prisoners was passing.'

## Create embeddings object

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

# embeddings
embedding_model = "sentence-transformers/all-MiniLM-l6-v2"
embeddings_folder = "/content/"

embeddings = HuggingFaceEmbeddings(model_name=embedding_model,
                                   cache_folder=embeddings_folder)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Test Embedding

In [ ]:
test_text = "Why do data scientists make great comedians? They're always trying to make ANOVA pun"
query_result = embeddings.embed_query(test_text)
query_result[:10]

[0.009409730322659016,
 -0.023806262761354446,
 -0.012127556838095188,
 0.036123793572187424,
 -0.03382446989417076,
 -0.07974203675985336,
 0.07004599273204803,
 0.0746554508805275,
 0.040141817182302475,
 0.044190675020217896]

In [ ]:
characters = len(test_text)
dimensions = len(query_result)
print(f"The {characters} character sentence was transformed into a {dimensions} dimension vector")

The 84 character sentence was transformed into a 384 dimension vector


# Create FAISS Vector Store

In [ ]:
from langchain.vectorstores import FAISS

## create FAISS Vector Store for ISLP

In [ ]:
book1_vs = FAISS.from_documents(islp_pages, embeddings)

## create FAISS Vector Store for Frankenstein

In [ ]:
book2_vs = FAISS.from_documents(frankenstein_docs, embeddings)

## create FAISS Vector Store for 1984

In [ ]:
book3_vs = FAISS.from_documents(nineteen_eighty_four_docs, embeddings)

# Save FAISS Vector Store

In [ ]:
# Save ISLP vector_store in project folder
path = '/content/drive/MyDrive/Final_Project/data/faiss_index_book1_ISLP'
book1_vs.save_local(path)

In [ ]:
# Save Frankenstein vector_store in project folder
path = '/content/drive/MyDrive/Final_Project/data/faiss_index_book2_frankenstein'
book2_vs.save_local(path)

In [ ]:
# Save 1984 vector_store in project folder
path = '/content/drive/MyDrive/Final_Project/data/faiss_index_book3_1984'
book3_vs.save_local(path)